In [ ]:
pip install tensorflow numpy pandas scikit-learn


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split


In [ ]:
# Number of samples
num_samples = 1000

# Generate synthetic user health data
np.random.seed(42)
data = {
    "age": np.random.randint(18, 60, num_samples),
    "gender": np.random.choice(["Male", "Female"], num_samples),
    "height_cm": np.random.randint(150, 200, num_samples),
    "weight_kg": np.random.randint(50, 120, num_samples),
    "bmi": np.random.uniform(18, 35, num_samples),
    "body_fat_percent": np.random.uniform(10, 40, num_samples),
    "muscle_mass_kg": np.random.uniform(40, 80, num_samples),
    "bone_mass_kg": np.random.uniform(2, 5, num_samples),
    "water_percent": np.random.uniform(50, 70, num_samples),
    "bmr_kcal": np.random.randint(1200, 2500, num_samples),
    "visceral_fat": np.random.randint(1, 15, num_samples),
    "metabolic_age": np.random.randint(18, 55, num_samples),
    "activity_level": np.random.choice(["Sedentary", "Moderate", "Active"], num_samples),
    "fitness_goal": np.random.choice(["Weight Loss", "Muscle Gain", "Maintain"], num_samples)
}

df = pd.DataFrame(data)


In [ ]:
df["gender"] = df["gender"].map({"Male": 0, "Female": 1})
df["activity_level"] = df["activity_level"].map({"Sedentary": 0, "Moderate": 1, "Active": 2})
df["fitness_goal"] = df["fitness_goal"].map({"Weight Loss": 0, "Muscle Gain": 1, "Maintain": 2})


In [ ]:
X = df.drop(columns=["fitness_goal"])  # Input features
y_diet = np.random.randint(1800, 3500, (num_samples,))  # Diet Plan: Calories per day
y_workout = np.random.randint(3, 6, (num_samples,))  # Workout Plan: Days per week


In [ ]:
X_train, X_test, y_diet_train, y_diet_test, y_workout_train, y_workout_test = train_test_split(
    X, y_diet, y_workout, test_size=0.2, random_state=42
)


In [ ]:
# Input layer
input_layer = Input(shape=(X_train.shape[1],))

# Hidden layers
x = Dense(64, activation='relu')(input_layer)
x = Dense(32, activation='relu')(x)

# Output 1: Diet Plan (Calories)
output_diet = Dense(1, activation='linear', name="diet_output")(x)

# Output 2: Workout Plan (Days per week)
output_workout = Dense(1, activation='linear', name="workout_output")(x)

# Create the model
model = keras.Model(inputs=input_layer, outputs=[output_diet, output_workout])

# Compile the model
model.compile(optimizer='adam', loss={'diet_output': 'mse', 'workout_output': 'mse'})

# Summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 13)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │            896 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 32)             │          2,080 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ diet_output (Dense)       │ (None, 1)              │             33 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ workout_output (Dense)    │ (None, 1)              │             33 │ dense_1[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,042 (11.88 KB)

 Trainable params: 3,042 (11.88 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    X_train, {"diet_output": y_diet_train, "workout_output": y_workout_train},
    validation_data=(X_test, {"diet_output": y_diet_test, "workout_output": y_workout_test}),
    epochs=50, batch_size=16
)


Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - diet_output_loss: 7328224.0000 - loss: 7329055.0000 - workout_output_loss: 831.0612 - val_diet_output_loss: 3785425.2500 - val_loss: 3785895.0000 - val_workout_output_loss: 167.6687
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - diet_output_loss: 2716151.7500 - loss: 2716206.7500 - workout_output_loss: 55.2069 - val_diet_output_loss: 496993.1250 - val_loss: 498501.1562 - val_workout_output_loss: 9.6082
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - diet_output_loss: 460493.7812 - loss: 460500.3438 - workout_output_loss: 6.5556 - val_diet_output_loss: 450758.3125 - val_loss: 451206.8750 - val_workout_output_loss: 8.7126
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - diet_output_loss: 451782.2812 - loss: 451788.6250 - workout_output_loss: 6.3328 - val_diet_output_loss: 441983.5625 - val_loss: 441970.2812 - val_workout_output_loss: 5.1085
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - diet_output_loss: 441655.7188 - 

In [ ]:
sample_user = np.array([[
    25,   # Age
    0,    # Gender (0 = Male, 1 = Female)
    175,  # Height (cm)
    70,   # Weight (kg)
    22.5, # BMI
    18.0, # Body Fat %
    60,   # Muscle Mass (kg)
    3.2,  # Bone Mass (kg)
    55,   # Water %
    1800, # BMR (kcal)
    5,    # Visceral Fat
    30,   # Metabolic Age
    1,    # Activity Level (0 = Sedentary, 1 = Moderate, 2 = Active)
]])


In [ ]:
# Make prediction
predicted_diet, predicted_workout = model.predict(sample_user)

# Convert predictions to integers for easy readability
predicted_diet = int(predicted_diet[0][0])
predicted_workout = int(predicted_workout[0][0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step


In [ ]:
import pandas as pd

# Create a DataFrame for better readability
output_df = pd.DataFrame({
    "Plan": ["Diet Plan", "Workout Plan"],
    "Calories / Days": [f"{predicted_diet} kcal per day", f"{predicted_workout} workout days per week"]
})

# Display the table
print(output_df)


           Plan          Calories / Days
0     Diet Plan        2509 kcal per day
1  Workout Plan  2 workout days per week


In [ ]:
import pandas as pd
from tabulate import tabulate

# Meal Plan Data
meal_data = [
    ["Breakfast (8:00 AM)", "Oats with nuts & banana", 350, "Whole-wheat toast with peanut butter (380 kcal)"],
    ["", "Scrambled egg whites & toast", 400, "Boiled eggs with avocado (420 kcal)"],
    ["", "Smoothie (protein, berries, flaxseeds)", 450, "Yogurt with granola & fruits (430 kcal)"],
    ["Mid-Morning Snack (10:30 AM)", "Greek yogurt with berries", 200, "Cottage cheese with flaxseeds (220 kcal)"],
    ["", "Boiled eggs & almonds", 250, "Protein bar (260 kcal)"],
    ["Lunch (1:00 PM)", "Grilled chicken, brown rice & veggies", 600, "Tofu stir-fry with brown rice (550 kcal)"],
    ["", "Lentils, quinoa & spinach salad", 650, "Chickpea salad with feta (620 kcal)"],
    ["Evening Snack (4:30 PM)", "Handful of mixed nuts", 200, "Trail mix with dark chocolate (210 kcal)"],
    ["", "Hummus with carrot & celery sticks", 180, "Guacamole with whole-wheat pita (190 kcal)"],
    ["Dinner (8:00 PM)", "Salmon with quinoa & greens", 500, "Grilled chicken with sweet potatoes (520 kcal)"],
    ["", "Tofu stir-fry with brown rice", 550, "Paneer curry with whole-wheat roti (530 kcal)"],
]

# Workout Plan Data
workout_data = [
    ["Day 1", "Strength Training (Upper Body)", "Bench Press", "3 sets of 10 reps", 150, "Rest 60s"],
    ["", "", "Bicep Curls", "3 sets of 15 reps", 100, "Rest 30s"],
    ["", "", "Triceps Dips", "3 sets of 12 reps", 120, "Rest 45s"],
    ["Day 2", "Cardio", "Running", "30 minutes", 400, "Rest 5 min"],
    ["", "", "Jump Rope", "20 minutes", 350, "Rest 3 min"],
    ["Day 3", "HIIT", "Burpees", "3 sets of 15 reps", 200, "Rest 30s"],
    ["", "", "Plank Jacks", "3 sets of 20 sec", 120, "Rest 30s"],
    ["Day 4", "Gymnastics", "Handstands", "10 minutes", 180, "Rest 1 min"],
    ["", "", "Backflips", "15 minutes", 220, "Rest 2 min"],
    ["Day 5", "Outdoor Sports", "Football", "40 minutes", 450, "Rest 5 min"],
    ["", "", "Swimming", "30 minutes", 500, "Rest 5 min"],
    ["Day 6", "Acrobatics", "Trampoline Jumps", "20 minutes", 250, "Rest 3 min"],
    ["", "", "Rope Climbing", "10 minutes", 300, "Rest 3 min"],
    ["Day 7", "Rest Day", "Complete Recovery", "-", "-", "-"]
]

# Creating Pandas DataFrames
meal_df = pd.DataFrame(meal_data, columns=["Time", "Meal Item", "Calories (kcal)", "Alternative"])
workout_df = pd.DataFrame(workout_data, columns=["Day", "Workout Type", "Exercise", "Reps/Duration", "Calories Burned", "Rest Time"])

# Display the tables in tabular format
print("🛒 **Meal Plan:**\n")
print(tabulate(meal_df, headers='keys', tablefmt='grid'))

print("\n🏋️ **Workout Plan:**\n")
print(tabulate(workout_df, headers='keys', tablefmt='grid'))


🛒 **Meal Plan:**

+----+------------------------------+----------------------------------------+-------------------+-------------------------------------------------+
|    | Time                         | Meal Item                              |   Calories (kcal) | Alternative                                     |
+====+==============================+========================================+===================+=================================================+
|  0 | Breakfast (8:00 AM)          | Oats with nuts & banana                |               350 | Whole-wheat toast with peanut butter (380 kcal) |
+----+------------------------------+----------------------------------------+-------------------+-------------------------------------------------+
|  1 |                              | Scrambled egg whites & toast           |               400 | Boiled eggs with avocado (420 kcal)             |
+----+------------------------------+----------------------------------------+----------

In [ ]:
# Example input with allergy flag (1 means allergic, 0 means not allergic)
sample_input = np.array([[70, 22, 15, 40, 5, 14, 55, 2200, 1]])  # 1 = allergic to nuts


In [ ]:
from prettytable import PrettyTable

# Example User Allergies (Modify based on user input)
user_allergy = "nuts"

# Meal Plan with Allergy Consideration
meal_plan = PrettyTable()
meal_plan.field_names = ["Meal", "Food", "Calories", "Alternative", "Allergy-Safe?"]

meals = [
    ("Breakfast", "Oatmeal + Nuts", "350 kcal", "Whole Wheat Toast", "nuts"),
    ("Lunch", "Grilled Chicken + Peanut Sauce", "600 kcal", "Tofu + Salad", "nuts"),
    ("Dinner", "Fish + Almond Quinoa", "500 kcal", "Lentil Soup + Rice", "nuts"),
]

for meal in meals:
    meal_name, food, cal, alt, allergy = meal
    if user_allergy in allergy:
        meal_plan.add_row([meal_name, alt, cal, "-", "✅ Safe"])
    else:
        meal_plan.add_row([meal_name, food, cal, alt, "✅ Safe"])

# Display the updated meal plan
print("\n🔹 Allergy-Safe Meal Plan:")
print(meal_plan)



🔹 Allergy-Safe Meal Plan:
+-----------+--------------------+----------+-------------+---------------+
|    Meal   |        Food        | Calories | Alternative | Allergy-Safe? |
+-----------+--------------------+----------+-------------+---------------+
| Breakfast | Whole Wheat Toast  | 350 kcal |      -      |    ✅ Safe    |
|   Lunch   |    Tofu + Salad    | 600 kcal |      -      |    ✅ Safe    |
|   Dinner  | Lentil Soup + Rice | 500 kcal |      -      |    ✅ Safe    |
+-----------+--------------------+----------+-------------+---------------+


In [ ]:
# Example input: [weight, bmi, muscle_mass, fat_mass, visceral_fat, protein, age, calories_needed, allergy, preference]
sample_input = np.array([[70, 22, 15, 40, 5, 14, 55, 2200, "nuts", "high-protein"]])


In [ ]:
from prettytable import PrettyTable

# Example User Preferences & Allergies
user_allergy = "nuts"
user_preference = "high-protein"

# Meal Plan with Preferences & Allergies
meal_plan = PrettyTable()
meal_plan.field_names = ["Meal", "Food", "Calories", "Alternative", "Preference Match?", "Allergy-Safe?"]

meals = [
    ("Breakfast", "Oatmeal + Nuts", "350 kcal", "Whole Wheat Toast", "✅ High-Protein", "nuts"),
    ("Lunch", "Grilled Chicken + Peanut Sauce", "600 kcal", "Tofu + Salad", "✅ High-Protein", "nuts"),
    ("Dinner", "Fish + Almond Quinoa", "500 kcal", "Lentil Soup + Rice", "✅ High-Protein", "nuts"),
    ("Snack", "Greek Yogurt + Berries", "200 kcal", "-", "✅ High-Protein", "None"),
]

for meal in meals:
    meal_name, food, cal, alt, preference_match, allergy = meal

    # Check for allergies
    if user_allergy in allergy:
        safe_food = alt
        allergy_safe = "✅ Safe"
    else:
        safe_food = food
        allergy_safe = "✅ Safe"

    # Check if preference matches
    preference_match = "✅" if user_preference in preference_match else "❌"

    meal_plan.add_row([meal_name, safe_food, cal, alt, preference_match, allergy_safe])

# Display the updated meal plan
print("\n🔹 Personalized Meal Plan (Considering Allergies & Preferences):")
print(meal_plan)



🔹 Personalized Meal Plan (Considering Allergies & Preferences):
+-----------+------------------------+----------+--------------------+-------------------+---------------+
|    Meal   |          Food          | Calories |    Alternative     | Preference Match? | Allergy-Safe? |
+-----------+------------------------+----------+--------------------+-------------------+---------------+
| Breakfast |   Whole Wheat Toast    | 350 kcal | Whole Wheat Toast  |         ❌        |    ✅ Safe    |
|   Lunch   |      Tofu + Salad      | 600 kcal |    Tofu + Salad    |         ❌        |    ✅ Safe    |
|   Dinner  |   Lentil Soup + Rice   | 500 kcal | Lentil Soup + Rice |         ❌        |    ✅ Safe    |
|   Snack   | Greek Yogurt + Berries | 200 kcal |         -          |         ❌        |    ✅ Safe    |
+-----------+------------------------+----------+--------------------+-------------------+---------------+


In [ ]:
# Example User Goal
user_goal = "muscle-gain"

workout_plan = PrettyTable()
workout_plan.field_names = ["Exercise", "Type", "Duration", "Calories Burned"]

workouts = [
    ("Bench Press", "Strength", "4 sets x 8 reps", "250 kcal"),
    ("Deadlifts", "Strength", "4 sets x 6 reps", "300 kcal"),
    ("Cycling", "Cardio", "30 mins", "400 kcal"),
    ("Jump Rope", "Cardio", "15 mins", "150 kcal"),
]

for workout in workouts:
    exercise, type_, duration, cal = workout

    if user_goal == "muscle-gain" and type_ == "Strength":
        workout_plan.add_row([exercise, type_, duration, cal])
    elif user_goal == "weight-loss" and type_ == "Cardio":
        workout_plan.add_row([exercise, type_, duration, cal])
    elif user_goal == "general-fitness":
        workout_plan.add_row([exercise, type_, duration, cal])

print("\n🔹 Personalized Workout Plan:")
print(workout_plan)



🔹 Personalized Workout Plan:
+-------------+----------+-----------------+-----------------+
|   Exercise  |   Type   |     Duration    | Calories Burned |
+-------------+----------+-----------------+-----------------+
| Bench Press | Strength | 4 sets x 8 reps |     250 kcal    |
|  Deadlifts  | Strength | 4 sets x 6 reps |     300 kcal    |
+-------------+----------+-----------------+-----------------+


In [ ]:
import sqlite3

# Connect to SQLite database (or create one if it doesn’t exist)
conn = sqlite3.connect("user_feedback.db")
cursor = conn.cursor()

# Create table to store feedback
cursor.execute('''
CREATE TABLE IF NOT EXISTS feedback (
    user_id INTEGER,
    meal TEXT,
    satisfaction INTEGER,
    skipped INTEGER,
    fullness INTEGER
)
''')

conn.commit()
conn.close()


In [ ]:
# Function to Adjust Meal Plan
def adjust_meal_plan(user_id):
    conn = sqlite3.connect("user_feedback.db")
    cursor = conn.cursor()

    # Get feedback from the user
    cursor.execute("SELECT meal, satisfaction, skipped, fullness FROM feedback WHERE user_id = ?", (user_id,))
    feedback_data = cursor.fetchall()

    adjusted_meals = []

    for meal, satisfaction, skipped, fullness in feedback_data:
        if skipped == 1:
            new_meal = "Alternate Meal"
        elif fullness < 3:
            new_meal = meal + " + Extra Portion"
        else:
            new_meal = meal

        adjusted_meals.append(new_meal)

    conn.close()
    return adjusted_meals

# Example: Get new meal plan after user feedback
user_id = 101
new_meal_plan = adjust_meal_plan(user_id)
print("Updated Meal Plan:", new_meal_plan)


Updated Meal Plan: []


In [ ]:
import pandas as pd

# Sample feedback data from users
feedback_data = {
    "user_id": [101, 102, 103],
    "meal": ["Oatmeal with Almonds", "Grilled Chicken", "Salmon Salad"],
    "satisfaction": [2, 5, 3],   # 1 (Disliked) → 5 (Loved)
    "skipped": [1, 0, 0],        # 1 = Skipped, 0 = Ate
    "fullness": [2, 4, 3],       # 1 (Still Hungry) → 5 (Full)
    "workout": ["HIIT", "Weightlifting", "Cycling"],
    "workout_difficulty": [4, 3, 5]  # 1 (Too Easy) → 5 (Too Hard)
}

# Convert to DataFrame
df = pd.DataFrame(feedback_data)

# Function to adjust meal plan
def adjust_meal_plan(df):
    adjusted_meals = []
    adjusted_workouts = []

    for _, row in df.iterrows():
        meal = row["meal"]
        workout = row["workout"]

        # Adjust meal plan
        if row["skipped"] == 1:
            new_meal = "Alternative Meal"
        elif row["fullness"] < 3:
            new_meal = meal + " + Extra Portion"
        else:
            new_meal = meal

        adjusted_meals.append(new_meal)

        # Adjust workout plan
        if row["workout_difficulty"] > 4:
            new_workout = workout + " (Lower Intensity)"
        elif row["workout_difficulty"] < 2:
            new_workout = workout + " (Higher Intensity)"
        else:
            new_workout = workout

        adjusted_workouts.append(new_workout)

    df["adjusted_meal"] = adjusted_meals
    df["adjusted_workout"] = adjusted_workouts
    return df

# Generate the adjusted plan
adjusted_df = adjust_meal_plan(df)
print(adjusted_df)


   user_id                  meal  satisfaction  skipped  fullness  \
0      101  Oatmeal with Almonds             2        1         2   
1      102       Grilled Chicken             5        0         4   
2      103          Salmon Salad             3        0         3   

         workout  workout_difficulty     adjusted_meal  \
0           HIIT                   4  Alternative Meal   
1  Weightlifting                   3   Grilled Chicken   
2        Cycling                   5      Salmon Salad   

            adjusted_workout  
0                       HIIT  
1              Weightlifting  
2  Cycling (Lower Intensity)  


In [ ]:
import pandas as pd

# Sample User Data with Preferences & Allergies
user_data = {
    "user_id": [101, 102, 103],
    "diet_type": ["Vegan", "Keto", "Vegetarian"],
    "allergies": [["Nuts", "Dairy"], ["Gluten"], ["Soy"]],
    "workout_preference": ["Outdoor", "Gym", "Home"]
}

# Sample Meal Plan
meal_plan = {
    "meal": ["Oatmeal with Almonds", "Grilled Chicken", "Salmon Salad"],
    "ingredients": [["Oats", "Almonds", "Milk"], ["Chicken", "Butter", "Spices"], ["Salmon", "Soy Sauce", "Lettuce"]],
    "calories": [300, 450, 400]
}

# Sample Workout Plan
workout_plan = {
    "workout": ["Cycling", "Weightlifting", "Yoga"],
    "calories_burned": [250, 400, 200]
}

# Convert to DataFrame
users_df = pd.DataFrame(user_data)
meals_df = pd.DataFrame(meal_plan)
workouts_df = pd.DataFrame(workout_plan)

# Function to adjust meals based on user preferences
def adjust_meals(users_df, meals_df):
    adjusted_meals = []

    for _, user in users_df.iterrows():
        user_meals = []

        for i, row in meals_df.iterrows():
            meal_name = row["meal"]
            meal_ingredients = row["ingredients"]
            adjusted_meal = meal_name

            # Remove allergens
            for allergen in user["allergies"]:
                if allergen in meal_ingredients:
                    adjusted_meal = "Alternative Meal (No " + allergen + ")"

            # Adjust for


In [ ]:
import random

# Food database with alternatives (for different diets & allergens)
food_db = {
    "Oats": {"vegan": "Quinoa", "keto": "Scrambled Eggs", "calories": 150},
    "Almonds": {"vegan": "Pumpkin Seeds", "keto": "Walnuts", "calories": 200},
    "Milk": {"vegan": "Almond Milk", "keto": "Coconut Milk", "calories": 100},
    "Chicken": {"vegan": "Tofu", "keto": "Salmon", "calories": 250},
    "Salmon": {"vegan": "Chickpeas", "keto": "Steak", "calories": 300},
    "Rice": {"vegan": "Brown Rice", "keto": "Cauliflower Rice", "calories": 200},
}

# Function to find alternatives
def find_alternative(food, diet):
    if food in food_db:
        return food_db[food].get(diet, food)  # Get diet-specific alternative
    return food  # Default to same food

# Function to generate AI-optimized meal plan
def generate_meal_plan(user):
    meals = ["Breakfast", "Lunch", "Dinner"]
    optimized_plan = []

    for meal in meals:
        original_food = random.choice(list(food_db.keys()))  # Select random food
        alternative = find_alternative(original_food, user["diet_type"].lower())

        optimized_plan.append({
            "Meal": meal,
            "Original Food": original_food,
            "Alternative": alternative,
            "Calories": food_db[original_food]["calories"]
        })

    return optimized_plan

# Example user
user_1 = {"diet_type": "Vegan", "allergies": ["Dairy", "Nuts"]}

# Generate optimized meal plan
meal_plan = generate_meal_plan(user_1)

# Display
import pandas as pd
df_meal = pd.DataFrame(meal_plan)
print(df_meal)


        Meal Original Food    Alternative  Calories
0  Breakfast          Rice     Brown Rice       200
1      Lunch       Almonds  Pumpkin Seeds       200
2     Dinner          Rice     Brown Rice       200


In [ ]:
# Workout database
workout_db = {
    "Gym": [
        {"Workout": "Weightlifting", "Calories Burned": 400},
        {"Workout": "Treadmill", "Calories Burned": 300}
    ],
    "Outdoor": [
        {"Workout": "Cycling", "Calories Burned": 250},
        {"Workout": "Running", "Calories Burned": 350}
    ],
    "Home": [
        {"Workout": "Yoga", "Calories Burned": 200},
        {"Workout": "Bodyweight Exercises", "Calories Burned": 300}
    ]
}

# Function to generate workout plan
def generate_workout_plan(user, goal):
    plan = []

    # Adjust workout type based on fitness goal
    if goal == "Weight Loss":
        intensity_factor = 1.2  # Burn more calories
    elif goal == "Muscle Gain":
        intensity_factor = 0.9  # Focus more on strength
    else:
        intensity_factor = 1.0  # Balanced workouts

    for workout in workout_db[user["workout_preference"]]:
        plan.append({
            "Workout": workout["Workout"],
            "Calories Burned": int(workout["Calories Burned"] * intensity_factor),
            "Rest Time (mins)": 45 if "Weightlifting" in workout["Workout"] else 30
        })

    return plan

# Example user and goal
user_1["workout_preference"] = "Gym"
fitness_goal = "Weight Loss"

# Generate workout plan
workout_plan = generate_workout_plan(user_1, fitness_goal)

# Display
df_workout = pd.DataFrame(workout_plan)
print(df_workout)


         Workout  Calories Burned  Rest Time (mins)
0  Weightlifting              480                45
1      Treadmill              360                30


In [ ]:
import random

# Function to generate synthetic user health data
def generate_user_data():
    return {
        "Weight (kg)": round(random.uniform(50, 100), 1),
        "BMI": round(random.uniform(18, 30), 1),
        "Body Fat (%)": round(random.uniform(10, 30), 1),
        "Muscle Mass (kg)": round(random.uniform(40, 80), 1),
        "Metabolism (kcal/day)": round(random.uniform(1400, 2200), 1),
        "Hydration (%)": round(random.uniform(45, 65), 1),
        "Visceral Fat": round(random.uniform(1, 15), 1),
        "Bone Mass (kg)": round(random.uniform(2.5, 4.5), 1),
    }

# Generate synthetic data
user_health_data = generate_user_data()

# Display
import pandas as pd
df_health = pd.DataFrame([user_health_data])
print(df_health)


   Weight (kg)   BMI  Body Fat (%)  Muscle Mass (kg)  Metabolism (kcal/day)  \
0         61.3  19.9          23.8              74.5                 2019.9   

   Hydration (%)  Visceral Fat  Bone Mass (kg)  
0           51.1           4.3             3.5  


In [ ]:
# User input
user_preferences = {
    "diet_type": "Keto",  # Options: Vegan, Vegetarian, Keto, Regular
    "allergies": ["Dairy", "Nuts"]  # Allergens to avoid
}


In [ ]:
# Workout database
workout_db = [
    ["Weightlifting", "Squats", "3 sets of 10 reps", 30, 250],
    ["Cardio", "Treadmill", "30 mins", 30, 360],
    ["Gymnastics", "Handstands", "3 sets of 15 sec hold", 20, 100],
    ["Outdoor Sport", "Basketball", "40 mins", 60, 450]
]

# Convert to DataFrame
df_workout = pd.DataFrame(workout_db, columns=["Category", "Exercise", "Reps/Sets", "Rest Time (min)", "Calories Burned"])
print(tabulate(df_workout, headers='keys', tablefmt='grid'))


+----+---------------+------------+-----------------------+-------------------+-------------------+
|    | Category      | Exercise   | Reps/Sets             |   Rest Time (min) |   Calories Burned |
+====+===============+============+=======================+===================+===================+
|  0 | Weightlifting | Squats     | 3 sets of 10 reps     |                30 |               250 |
+----+---------------+------------+-----------------------+-------------------+-------------------+
|  1 | Cardio        | Treadmill  | 30 mins               |                30 |               360 |
+----+---------------+------------+-----------------------+-------------------+-------------------+
|  2 | Gymnastics    | Handstands | 3 sets of 15 sec hold |                20 |               100 |
+----+---------------+------------+-----------------------+-------------------+-------------------+
|  3 | Outdoor Sport | Basketball | 40 mins               |                60 |               450 |
